In [1]:
import requests
import pandas as pd
import numpy as np

pd.options.display.max_columns = None

In [2]:
url = 'https://fantasy.premierleague.com/api/bootstrap-static/'
r = requests.get(url)
json = r.json()
elements_df = pd.DataFrame(json['elements'])
elements_types_df = pd.DataFrame(json['element_types'])
teams_df = pd.DataFrame(json['teams'])

In [3]:
for x in elements_df.index :
    #print(x)
    element_id = elements_df.id[x]
    first_name = elements_df.first_name[x]
    second_name = elements_df.second_name[x]
    name = first_name+" "+second_name
    team = elements_df.team[x]
    element_type = elements_df.element_type[x]
    url = f'https://fantasy.premierleague.com/api/element-summary/{element_id}/'
    r = requests.get(url)
    json = r.json()
    json_history_df = pd.DataFrame(json['history'])
    json_history_df['name'] = name
    json_history_df['team'] = team
    json_history_df['element_type'] = element_type
 
    
    json_history_past_df = pd.DataFrame(json['history_past'])
    json_history_past_df['name'] = name
    json_history_past_df['team'] = team
    json_history_past_df['element_type'] = element_type
    
    if x == 0 :
        all_history_df = json_history_df
        all_history_past_df = json_history_past_df
    else : 
        all_history_df = all_history_df.append(json_history_df)
        all_history_past_df = all_history_past_df.append(json_history_past_df)

In [4]:
all_history_df['position'] = all_history_df.element_type.map(elements_types_df.set_index('id').singular_name)
all_history_df['team'] = all_history_df.team.map(teams_df.set_index('id').name)

all_history_past_df['position'] = all_history_past_df.element_type.map(elements_types_df.set_index('id').singular_name)
all_history_past_df['team'] = all_history_past_df.team.map(teams_df.set_index('id').name)

In [5]:
all_history_df.to_csv('all_history_df_current.csv')
all_history_past_df.to_csv('all_history_past_df_historic.csv')